# <span >**Tic Tac Toe AI 🎮**</span>

### <span style="color: #2196F3;">A Smart and Interactive Tic Tac Toe Game with AI 🤖</span>
*Developed using Python and Pygame, featuring AI that plays as an unbeatable opponent!*

---

- <span style="color: #FF5722;">**Game Features:**</span>
  - Human vs AI 🧑‍🤝‍🧑
  - Minimax Algorithm for AI decision making 🧠
  - Beautiful UI with clean design ✨
  - Option to restart the game 🔄

---
### <span style="color: #FF9800;">**Ready to challenge the AI? 🏆**</span>




# Import necessary libraries

In [10]:
import numpy as np
import pygame
import sys

In [11]:
!pip install pygame

In [ ]:
# Constants pre-defined to make it easier to control and clean code
    # colors
Light_Gray = (200, 200, 200)   # Used for grid lines
Dark_Gray = (30, 30, 30)         # Used to fill the background
Gold = (255, 215, 0)         # Used to draw the win line or text
Sky_Blue = (0, 200, 255)       # O symbol
Light_Red = (255, 100, 100)        # X symbol

# red   green   blue  
# (255 , 255 , 255)

    # Board settings
WIDTH = 600              # Total width of the game window
HEIGHT = 600             # Total height of the game window
LINE_WIDTH = 10          # Width of grid lines
BOARD_ROWS = 3
BOARD_COLS = 3
SQUARE_SIZE = WIDTH // BOARD_COLS  # Each square size = 200 (600 / 3)
CIRCLE_RADIUS = SQUARE_SIZE // 3   # Radius of the O symbol
CIRCLE_WIDTH = 20                  # Thickness of the circle
CROSS_WIDTH = 30                   # Thickness of the X lines
WIN_LINE_WIDTH = 15                # Thickness of the win line



# ..................................................................................... #
# Initialize pygame
pygame.init()  # Starts Pygame engine
screen = pygame.display.set_mode((WIDTH, HEIGHT))  # Creates a 600x600 window
pygame.display.set_caption("Tic Tac Toe AI")  # Sets window title
screen.fill(Dark_Gray)  # Fills background with Dark_Gray

# Font sizes
font = pygame.font.Font(None, 75)  # Big font for symbols or headings
game_over_font = pygame.font.Font(None, 50)  # Font for "Game Over" message


# Game board array (store the game state after) , and variables to detect the winner
board = np.zeros((BOARD_ROWS, BOARD_COLS))  # 3x3 grid with all zeros
player = 1        # 1 = human player, 2 = AI
game_over = False # To track if the game ended
win_line = None   # Will store line coordinates when someone wins
winner = None     # Will store who won


# Draw grid lines (split the screen into 9 squares)
def draw_lines():
    for row in range(1, BOARD_ROWS):
        pygame.draw.line(screen, Light_Gray, (0, row * SQUARE_SIZE), (WIDTH, row * SQUARE_SIZE), LINE_WIDTH)
        pygame.draw.line(screen, Light_Gray, (row * SQUARE_SIZE, 0), (row * SQUARE_SIZE, HEIGHT), LINE_WIDTH)



# Draw figures (X and O)
def draw_figures():
    for row in range(BOARD_ROWS):
        for col in range(BOARD_COLS):
            if board[row][col] == 1:
                # X
                pygame.draw.line(
                    screen,
                    Light_Red,
                    (col * SQUARE_SIZE + 30, row * SQUARE_SIZE + 30),
                    (col * SQUARE_SIZE + SQUARE_SIZE - 30, row * SQUARE_SIZE + SQUARE_SIZE - 30),
                    CROSS_WIDTH
                )
                pygame.draw.line(
                    screen,
                    Light_Red,
                    (col * SQUARE_SIZE + 30, row * SQUARE_SIZE + SQUARE_SIZE - 30),
                    (col * SQUARE_SIZE + SQUARE_SIZE - 30, row * SQUARE_SIZE + 30),
                    CROSS_WIDTH
                )
            elif board[row][col] == 2:
                #  O
                pygame.draw.circle(
                    screen,
                    Sky_Blue,
                    (col * SQUARE_SIZE + SQUARE_SIZE // 2, row * SQUARE_SIZE + SQUARE_SIZE // 2),
                    CIRCLE_RADIUS,
                    CIRCLE_WIDTH
                )





# Mark square with O or X 
def mark_square(row, col, player):
    board[row][col] = player  

# Check if a square is available or is filled
def is_available(row, col):
    return board[row][col] == 0

# Check if the board is full 
def is_board_full():
    return not (board == 0).any()

# Check for win and get winning line 
def check_win(player):
    global win_line
    # rows checking:
    for row in range(BOARD_ROWS):
        if np.all(board[row, :] == player):
            win_line = ((0, row * SQUARE_SIZE + SQUARE_SIZE // 2), (WIDTH, row * SQUARE_SIZE + SQUARE_SIZE // 2))
            return True
            
    # columns checking
    for col in range(BOARD_COLS):
        if np.all(board[:, col] == player):
            win_line = ((col * SQUARE_SIZE + SQUARE_SIZE // 2, 0), (col * SQUARE_SIZE + SQUARE_SIZE // 2, HEIGHT))
            return True
            
    # Check main diagonal
    if np.all(np.diag(board) == player):
        win_line = ((0, 0), (WIDTH, HEIGHT))
        return True
        
    # Check anti-diagonal
    if np.all(np.diag(np.fliplr(board)) == player):
        win_line = ((0, HEIGHT), (WIDTH, 0))
        return True
    return False

# Draw the winning line ... (good indicator for the winner)
def draw_win_line():
    if win_line:
        pygame.draw.line(screen, Gold, win_line[0], win_line[1], WIN_LINE_WIDTH)

# Restart the game 
def restart_game():
    """Resets the game board, player turn, and game status to start a new game """
    
    global board, player, game_over, win_line, winner
    board = np.zeros((BOARD_ROWS, BOARD_COLS))
    player = 1
    game_over = False
    win_line = None
    winner = None
    screen.fill(Dark_Gray)
    draw_lines()

# AI move using minimax

def minimax(board, depth, is_maximizing):
    """A decision-making algorithm used to determine the optimal move for the AI by evaluating all possible board states recursively.
       U can learn more about it here (https://www.neverstopbuilding.com/blog/minimax) """

    if check_win(2):
        return 1  # AI wins
    if check_win(1):
        return -1  # Player wins (You can never win 😎🚫🎮 )
    if is_board_full():
        return 0  # Draw

    if is_maximizing:
        best_score = float('-inf')
        for row in range(BOARD_ROWS):
            for col in range(BOARD_COLS):
                if board[row][col] == 0:
                    board[row][col] = 2
                    score = minimax(board, depth + 1, False)
                    board[row][col] = 0
                    best_score = max(score, best_score)
        return best_score
    else:
        best_score = float('inf')
        for row in range(BOARD_ROWS):
            for col in range(BOARD_COLS):
                if board[row][col] == 0:
                    board[row][col] = 1
                    score = minimax(board, depth + 1, True)
                    board[row][col] = 0
                    best_score = min(score, best_score)
        return best_score

# According to the minimax algorithm we did .. we can now choose the best move
def best_move():
    best_score = float('-inf')
    move = None
    for row in range(BOARD_ROWS):
        for col in range(BOARD_COLS):
            if board[row][col] == 0:
                board[row][col] = 2
                score = minimax(board, 0, False)
                board[row][col] = 0
                if score > best_score:
                    best_score = score
                    move = (row, col)
    if move:
        mark_square(move[0], move[1], 2)

# Draw the initial grid after we initialized the screen
draw_lines()

# Game loop (main)
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

        if not game_over and player == 1 and event.type == pygame.MOUSEBUTTONDOWN:
            mouseX = event.pos[0] // SQUARE_SIZE
            mouseY = event.pos[1] // SQUARE_SIZE

            if is_available(mouseY, mouseX):
                mark_square(mouseY, mouseX, player)
                if check_win(player):
                    winner = "Player X"
                    game_over = True
                player = 2

                # Redraw the board after the player's move
                screen.fill(Dark_Gray)
                draw_lines()
                draw_figures()
                pygame.display.update()

                # Now let AI make its move after a brief delay
                pygame.time.wait(500)  # Adds a small delay before AI move

        if not game_over and player == 2:
            best_move()
            if check_win(2):
                winner = "AI"
                game_over = True
            player = 1

            # Redraw the board after the AI's move
            screen.fill(Dark_Gray)
            draw_lines()
            draw_figures()
            pygame.display.update()

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:
                restart_game()

    if game_over:
        draw_win_line()
        if winner:
            text = game_over_font.render(f"{winner} Wins!", True, Gold)
            screen.blit(text, (WIDTH // 4, HEIGHT // 2))
    pygame.display.update()
 
